오늘은 지금까지 배운 전반부 딥러닝에서의 내용을 총체적으로 활용하여 문제를 직접 a부터 z까지 해결해보는 시간입니다.

오늘 다룰 데이터는 names 안에 있는 각국의 이름을 받아, 이름의 국적을 추론하는 뉴럴 넷을 만드는 것입니다. 방법은 자유이나, 다음의 두 가지 방법을 꼭 포함시켜 구현하고, 성능 측정/튜닝의 과정을 거치세요.

1. Feedforward Network(MLP)를 사용하여 해볼 것.
  -  Feedforward Network를 이용하여 할 경우, input size가 이름에 따라 다른데 이를 어떤 식으로 다룰지 생각해 볼 것
2. RNN을 이용하여 해볼 것.
  - torch.DataLoader을 사용할 경우 batching을 하게 되는데, 이 경우 위 1에서 써 있는 문제와 비슷한 문제가 있으며 해결 또한 비슷하게 할 수 있음. torch.DataLoader을 쓸 때와 쓰지 않을 때를 비교해볼 것.
  - 어제는 영어 알파벳만 사용하였는데, 다른 알파벳까지 사용할 경우 성능이 올라가는지 내려가는지 관찰해볼 것.
  - 데이터에서 관찰하지 못한 알파벳들을 OOV로 처리하는 방식을 시도할 경우 성능이 올라가는지 내려가는지 관찰해볼 것

성능 측정을 위해서 train-valid-test를 적절히 나누고, valid에서의 loss와 정확도를 측정하면서 학습을 적절할 때 멈추거나 할 것.

튜닝을 위해서는 다음의 방법들을 사용할 수 있습니다.

- hyperparameter을 다양하게 시도해볼 것
- 뉴럴 넷 구조를 조금씩 바꿔볼 것
- 데이터를 더 만들어 볼 것

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
import random
import glob
import string

all_letters = string.ascii_lowercase + '?_'
category_names = {} # category_names: dict[str, list[str]]
all_categories = [] # all_categories: list[str]

def idx_tensor(idx, N):
    return torch.tensor([1 if i==idx else 0 for i in range(N)])

def name_to_tensor(name):
    res = torch.zeros(len(name), len(all_letters))
    for idx, letter in enumerate(name):
        res[idx][all_letters.find(letter)] = 1
    # print(name, res, res.shape)
    return res.squeeze(dim = 1)

def input_padding(word):
    return word + '_'*(10 - len(word)) if len(word)!=10 else word

def preprocessing_data(batch_size = 32):
    files = glob.glob('./data/names/*.txt')
    # print(f'{len(files)}fiels',*files,sep='\n')
    # max_length = []
    
    for file in files:
        with open(file) as f:
            words = f.read().strip().split('\n')
        lang = file.split('\\')[-1].split('.')[0]
        all_categories.append(lang)

        # print(len([n for n in map(lambda n: ''.join(c if c in all_letters else '?' for c in n.lower()), words) if len(n)<11]), end='=')
        # print(len([n for n in map(lambda n: ''.join(c if c in all_letters else '?' for c in n.lower()), words) if len(n)<15]))
        words = [input_padding(n) for n in map(lambda n: ''.join(c if c in string.ascii_lowercase else '?' for c in n.lower()), words) if len(n)<11]

        category_names[lang] = words
        # print(f'{lang}: {len(names)} |', *names[0:10])
    # print(max_length)
    # print(all_categories)
    # print(category_names)

    x,y = [],[]
    for lang, words in category_names.items():
        for name in words:
            x.append(name_to_tensor(name))
            y.append(idx_tensor( all_categories.index(lang), len(all_categories) ))
    
    dataset = TensorDataset(torch.stack(x), torch.stack(y))
    dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = True)
    return dataloader

def plot_loss_history(loss_history, val_loss_history=None): # 훈련손실, 검증손실
    plt.figure(figsize=(10, 6)) 
    plt.plot(range(1, len(loss_history) + 1), loss_history, label='Training Loss', color='blue', marker='o')
    if val_loss_history is not None: plt.plot(range(1, len(val_loss_history) + 1), val_loss_history, label='Validation Loss', color='orange', marker='x')
    plt.title('Loss History')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.xticks(range(1, len(loss_history) + 1)) # x축 눈금 설정
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


class RNN(nn.Module): # MLP
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.input_hidden = nn.Linear(input_size, hidden_size)
        self.hidden_hidden = nn.Linear(hidden_size, hidden_size)
        self.hidden_output = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU
        self.softmax = nn.LogSoftmax(dim = 0)
        self.optimizer = optim.Adam
        self.loss = torch.nn.NLLLoss()

    def forward(self, input, hidden):
        hidden = F.tanh(self.input_hidden(input) + self.hidden_hidden(hidden))
        output = self.hidden_output(hidden)
        # print('after h2o', output.shape)
        # output = self.relu(output)
        # print('after Relu', output.shape)
        output = self.softmax(output)
        # print('after softmax', output.shape)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(self.hidden_size) # 히든초기화

    def train_model(self, train_data, learning_rate = 0.001, epochs = 20):
        optimizer = self.optimizer(self.parameters(), lr = learning_rate)
        loss_history = []

        for epoch in range(epochs):
            running_loss = 0.0
            correct_predictions = 0
            total_predictions = 0
            
            for input, label in train_data:
                hidden = self.initHidden()
                print(input)
                print(label)
                
                for char in input:
                    output, hidden = self(char, hidden) # nn.Module 자체에 이러고 넣으면 forward를 호출해서 output과 hiden을 반환, 그냥 그렇다.
                    # print(output.shape)
                
                # print(output.shape, label.shape, input.shape)
                _, predicted = torch.max(output, 1)
                
                label_indices = torch.argmax(label, dim=1) # 레이블 인덱스 추출
                
                # 정확도 계산
                print(predicted.shape, label_indices.shape)
                correct_predictions += (predicted == label_indices).sum().item()
                total_predictions += label.size(0)
                
                loss = self.loss(output, label)
                loss_history.append(torch.log(torch.mean(loss)).item())
                # print(torch.mean(loss).item())

                if len(loss_history) % 1000 == 0:
                    print(torch.mean(loss).item())
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
            
            epoch_loss = running_loss / len(train_data)
            accuracy = correct_predictions / total_predictions  # 정확도 계산
            print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.5f}, Accuracy: {accuracy:.5f}')  # 정확도 출력
            plot_loss_history(loss_history)
        return loss_history

def predict_nationality(model, word):
    hidden = model.initHidden()
    word_tensor = name_to_tensor(input_padding(word))
    for char_tensor in word_tensor:
        output, hidden = rnn(char_tensor.unsqueeze(0), hidden)
    # print(output.shape)
    return output


dataset = preprocessing_data(batch_size=32)
n_letters = len(all_letters)
n_hidden = 32
n_categories = len(all_categories)

rnn = RNN(n_letters, n_hidden, n_categories)
predict_nationality(rnn, 'ang')

rnn.train_model(dataset, learning_rate = 0.001, epochs = 20)
print(1)

# Create a dataset and data loader # 데이터셋과 데이터로더 생성
# dataset = TensorDataset(X_tensor, y_tensor) # 입력 데이터(X_tensor)와 정답 데이터(y_tensor)로 텐서 데이터셋 생성
# batch_size = 32 # 한 번에 모델에 넣는 데이터 묶음 크기 (미니배치)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) # 배치로 데이터를 묶어서 모델에 전달할 수 있도록 데이터로더 생성, 셔플을 통해 데이터 순서를 무작위로 섞음

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.]],

        ...,

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

RuntimeError: The size of tensor a (10) must match the size of tensor b (32) at non-singleton dimension 0

In [ ]:
# 학습 데이터 준비
training_data = []
for category, words in category_names.items():
    category_index = category_to_index(category)
    for name in words:
        name_tensor = name_to_tensor(name)
        training_data.append((name_tensor, category_index))

In [ ]:

def letter2tensor(letter):
    [(res:=[1 if i==x else 0 for x in range(26)]) for i,j in enumerate(string.ascii_lowercase) if j==letter]
    return torch.tensor(res)

z_tensor = letter2tensor('z')
print(z_tensor.shape) # (26,1)
print(z_tensor)
# a_tensor = torch,tensor([1,0,...,0])

def word2tensor(word):
    res = torch.zeros(len(word), 1, len(all_letters))
    for idx, char in enumerate(word):
        res[idx] = letter2tensor(char)
    return res.squeeze(dim=1)
print(word2tensor('abc').shape)